# Splitting the dataset

In [3]:
import csv  # reading csv files
import pandas as pd # data science
import numpy as np  # data science
from sklearn.model_selection import train_test_split    # splitting the data
import itertools    # getting a list of all words from a dataframe

In [5]:
data = pd.read_csv("data/sample_preprocessed_no_punct.csv")

/var/folders/gj/r8x5s5mn7jg12hs3c28wpg4c0000gn/T/ipykernel_13688/1965015741.py:1: DtypeWarning: Columns (1,2,3,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/sample_preprocessed_no_punct.csv")


Split data before representing our data as bag of words for naive bayes classifier in order to avoid statistics leaking into validation and testing dat.

In [8]:
X_train, X_val, y_train, y_val = train_test_split(data['content'], data['type'], test_size=0.25, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0)

We define a function to create a bag of words table from a given dataframe

In [ ]:
def create_bag_of_words(dataframe):
    all_tokens = list(itertools.chain.from_iterable(dataframe['content']))